In [13]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

import time

warnings.filterwarnings('ignore')

In [14]:
from razor.api import project_space_path
import razor


In [15]:
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [16]:
TARGET = 'sales'         # Our main target
END_TRAIN = 1913         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [17]:
print('Load Main Data')

# We will need only train dataset
# to show lags concept
train_df = pd.read_csv(project_space_path('M5Forecasting/sales_train_validation.csv'))

# To make all calculations faster
# we will limit dataset by 'CA' state
train_df = train_df[train_df['state_id']=='CA']


Load Main Data


In [18]:
train_df.shape

(12196, 1919)

In [19]:
train_df.iloc[:10]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


In [ ]:
# index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
# train_df = pd.melt(train_df, 
#                   id_vars = index_columns, 
#                   var_name = 'd', 
#                   value_name = TARGET)

# train_df[train_df['id']=='HOBBIES_1_001_CA_1_validation'].iloc[:10]


In [ ]:
import razor.flow as rf
import typing as t
import logging
import pandas as pd
from razor.api import project_space_path


In [ ]:
# train_df.to_csv(project_space_path('M5Forecasting/train_df.csv'), index=False)



In [ ]:
@rf.block(executor=rf.ContainerExecutor(cores=2, memory=8192))
class HeavyOps():
    path: t.Any
        
    def run(self):

        train_df = pd.read_csv(project_space_path(path))
        
        # Or same in "compact" manner
        LAG_DAYS = [col for col in range(1,8)]
        temp_df = train_df[['id','d',TARGET]]

        start_time = time.time()
        temp_df = temp_df.assign(**{
                '{}_lag_{}'.format(col, l): temp_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
                for l in LAG_DAYS
                for col in [TARGET]
            })
        
        logging.info('%0.2f min: Time for bulk shift')
        logging.info(time.time() - start_time / 60)
        print('%0.2f min: Time for bulk shift' % ((time.time() - start_time) / 60))
        
        temp_df.to_csv(project_space_path('M5Forecasting/temp_df.csv'), index=False)
        

In [ ]:
heavy_ops = HeavyOps(path = 'M5Forecasting/train_df.csv')



In [ ]:
p = rf.Pipeline("Heavy Operation", targets=[heavy_ops])  #preprocess


In [ ]:
p.show()

In [ ]:
deployed_pipeline = razor.api.engines('DS-engine').execute(pipeline=p)


In [ ]:
deployed_pipeline

In [ ]:
temp_df = pd.read_csv('M5Forecasting/temp_df.csv')




# Original Lag Code

In [ ]:
@rf.block(executor=rf.ContainerExecutor(cores=2, memory=20000))
class LagCreator:
    grid_path: t.Any
        
    def run(self):
        ########################### Apply on grid_df

        grid_df = pd.read_pickle(project_space_path(self.grid_path))

        TARGET = 'sales'         # Our main target
        END_TRAIN = 1913         # Last day in train set
        MAIN_INDEX = ['id','d']  # We can identify item by these columns
        
        # We need only 'id','d','sales'
        # to make lags and rollings
        grid_df = grid_df[['id','d','sales']]
        SHIFT_DAY = 28

        # Lags
        # with 28 day shift
        start_time = time.time()
        self.logger.info('Create lags')

        LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
        grid_df = grid_df.assign(**{
                '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
                for l in LAG_DAYS
                for col in [TARGET]
            })

        # Minify lag columns
        for col in list(grid_df):
            if 'lag' in col:
                grid_df[col] = grid_df[col].astype(np.float16)

        self.logger.info('%0.2f min: Lags' % ((time.time() - start_time) / 60))

        # Rollings
        # with 28 day shift
        start_time = time.time()
        self.logger.info('Create rolling aggs')

        for i in [7,14,30,60,180]:
            self.logger.info(f'Rolling period: {i}')
            grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
            grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

        # Rollings
        # with sliding shift
        for d_shift in [1,7,14]: 
            self.logger.info(f'Shifting period: {d_shift}')
            for d_window in [7,14,30,60]:
                col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
                grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)


        self.logger.info('%0.2f min: Lags' % ((time.time() - start_time) / 60))

        self.logger.info(f'Save lags and rollings  {grid_df.shape}')
        grid_df.to_pickle(project_space_path(f'M5Forecasting/lags_df_{str(SHIFT_DAY)}.pkl'))


In [ ]:
lag_creator = LagCreator(grid_path = 'M5Forecasting/m5-simple-fe/grid_part_1.pkl')


In [ ]:
p = rf.Pipeline("lag_creator", targets=[lag_creator])  #preprocess


In [ ]:
p.show()

In [37]:
deployed_pipeline = razor.api.engines('DS-engine').execute(pipeline=p)


In [46]:
deployed_pipeline

PlatformPipelineRun(project_id='c0b26d85-a4c9-44f5-9d0e-f540dd2de644', pipeline_id='007f8806-0357-11eb-b88b-0242ac110004', pipeline_name='lag_creator', pipeline_run_id='5729c8de-bfe0-4e1d-a2c9-c4955878fa16', pipeline_version=None, comment=None, created_on='2020-09-30T19:56:17.662+00:00', start_time=None, end_time=None, eta=0, run_at=1601495777662, run_duration=0, compute_time=0, wait_time=37, ran_by_user=RanByUser(user_name='Ayan Basak', email='ayan.basak@razorthink.com'), status='IN_PROGRESS', block_status=[PlatformPipelineBlockRun(pipeline_run_id='5729c8de-bfe0-4e1d-a2c9-c4955878fa16', pipeline_name='lag_creator', pipeline_status='IN_PROGRESS', block_id='cb38302a-57a5-4d9c-9b60-8e11610bb429', block_run_id='f68703d1-f94a-4ed4-a4be-bf84776be6dc', block_name='LagCreator_1', resource_spec=ResourceAllocated(cores=2, memory=20000, use_gpu=False, gpu=0, use_gpu_if_available=False, cluster_configuration={'worker': None, 'workerCore': None, 'workerMemory': None, 'numberOfInstances': None, 'maxServerCount': None}, additional_env={}, run_env='PYTHON'), technology='PYTHON', status='READY_TO_GO', containers=[], cluster_id='cb38302a-57a5-4d9c-9b60-8e11610bb429', _log=None, _metric=None, wait_time=0, compute_time=0, start_time=None, end_time=None, queued_at='2020-09-30T19:56:18.000+00:00')], run_number=194, pipeline_variable_list=[], block_run_details=[BlockRunDetail(block_id='cb38302a-57a5-4d9c-9b60-8e11610bb429', block_run_id='f68703d1-f94a-4ed4-a4be-bf84776be6dc', block_name='LagCreator_1', technology='PYTHON', log_path=None, input_parameters='{\n  "grid_path": {\n    "dtype": "Any",\n    "required": "false",\n    "series": false,\n    "adapter": "eJxrYJmqxgABPYJFiVX5RXppOfnlern5Kak5xVN6ePLy8wviE1MSC0pSi6ZMnqIHAF/+ELQ=",\n    "validator": "eJxrYJmqwQABPYJFiVX5RXppOfnlern5Kak5xVN6+PLy8wviyxJzMlMSS/KLpkyeogcAg2wRnQ==",\n    "source_id": null,\n    "source_name": null,\n    "inferred_type": "STATIC",\n    "value": "eJxrYJmqxwABPVq+pm75RanJicUlmXnp+rmmusWZuQU5qbppqfrpRZkp8QWJRSXxhnoF2TlT9ADQohIn"\n  }\n}', output_parameters=None, status='READY_TO_GO', created_at='2020-09-30T19:56:17.758+00:00', queued_at='2020-09-30T19:56:18.000+00:00', start_time=None, end_time=None, compute_time=0, wait_time=0, run_time=0, block_run_output_list=[], resource_allocated=ResourceAllocated(cores=2, memory=20000, use_gpu=False, gpu=0, use_gpu_if_available=False, cluster_configuration={'worker': None, 'workerCore': None, 'workerMemory': None, 'numberOfInstances': None, 'maxServerCount': None}, additional_env={}, run_env='PYTHON'))], pipeline_json='{\n  "id": "007f8806-0357-11eb-b88b-0242ac110004",\n  "projectId": "c0b26d85-a4c9-44f5-9d0e-f540dd2de644",\n  "name": "lag_creator",\n  "description": "",\n  "blocks": [\n    {\n      "name": "LagCreator_1",\n      "id": "cb38302a-57a5-4d9c-9b60-8e11610bb429",\n      "artifact_id": null,\n      "class_name": "LagCreator",\n      "code": "eJyVV9tuG8cZ3lkuKUq2ZUl2dbBdh0gPoGuLkm1JPkQw6lBOYtNiA8mAY6DCYMhZclZa7lJ7sK2GCpL6IAoYGC28vakRRMoLtGjcoigKOL3NQ/Q+vekjpN/skqKc2G1KAtydf2b+w/cf+bHxO/uwFn/kWNV2Q960qmu2Wdj3HskJ6plig3ssMKm/Ztpm4Dq0ajPfj55GeaHJo7TB1l61J7OV0LIDy/EjaQQbTRNEOXCT1YueyQLXi4SeF0Ze9ImsSMuMW1k1qwHOPIo2I9lHKXerlEZlX+YqM3PTFy7NzHJ+/tzMubOzlbNsenpmrjZXqXF2YW4uKgfREm7l5QClDZeHtombMosFsxz1ephS5jhuwALLdXxQIKK/7lmcNlkgIpmBfpZTj2TqqrMBHXyZ8kIngnmDtGbZNq2FTlXd7Rh9uGd0vAUyqIdox2LaNTdbdLl5Sy0eQcN8iZS0Uh++qbd/rGtaoO2QQG9pq6kHZJdsG5xsE65vEp7iRl3f1NuGeuOknprQsJve1gJjx9jVNlO4k97J8MwuIRrv49mHWtDX1toa7x/THugL2gOyabSN1WxN2yJbRk3nA/zAw2zLUDtPIOn+maC/bTwgC9pKbjPND7bSaxnvYdtopSe0nYHgwOrBXaJW89r6w9taR9qhRHorNa7xwePaqKakf0ujw4p2/zEfAuePttMd6nA7/Uz7VFd7bWPnEE/VYW+bQNbglraVhn4j/MjD7C7pyeZHg8Pt9AMyps2/7s4PvnNntHdn/Z+3tfsBH4Me9rbe0WO8rXf1uL/EJ7B3YzvFj+GWrm7x42Ma3lPqfTPzHan6VgpST3xbaiszr/2GrP/6tvqt/Xe0Ons/hG+G9iEyHOj8JCTDO880/sancTzAs9qyFuXLMu0z2/SjxY0+qVs8koRHpRPyQJxHZs5mdT8SP0qCSy+lSsvFoTokrWQ3SYv8Vl83wOXUKakXphHfVdeOHkfyjXmruREI15m0nGYYTJ6fmZyuzbKL5tzZ8+Zc5Qoydd62/KDqNppXotJ4Uc8oJirP5Ju9HC4gSwrztltltn+lsP/G4Z5G2dKR4s8tpdHFba1N6irqzmyStr4DCx